In [1]:
import pandahouse as ph

In [2]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database': 'default',
                      'user': 'student', 
                      'password': 'dpo_python_2020'}

# Написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. 
Назовём очень усердными студентами тех пользователей, которые хотя бы раз за текущий месяц правильно решили 20 маленьких заданий.

# Что было сделано:
1. Смотрю на данные (предисследование данных таблицы peas):

In [3]:
df = '''
    select
        st_id,
        toDateTime(timest),
        correct,
        subject
    from default.peas
    limit 10
'''

ph.read_clickhouse(df, connection=connection_default)

,st_id,toDateTime(timest),correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability
3,100379,2021-10-30 16:44:50,1,Vizualization
4,100379,2021-10-30 17:15:05,1,Theory of probability
5,100379,2021-10-30 18:02:37,1,Theory of probability
6,100379,2021-10-30 18:17:25,1,Vizualization
7,100379,2021-10-30 18:32:26,0,Theory of probability
8,100379,2021-10-30 19:19:33,1,Vizualization
9,100379,2021-10-30 19:28:03,1,Theory of probability


2. В задаче фигурирует условие "за текущий месяц". 
Смотрю на уникальные даты (предисследование данных таблицы peas):

In [4]:
df_2 = '''
    SELECT toDate(timest) as new_date 
    FROM default.peas
    group by toDate(timest)
    limit 10
'''

ph.read_clickhouse(df_2, connection=connection_default)

,new_date
0,2021-10-30
1,2021-10-31


В таблице только 2 дня в октябре 2021: 30 и 31 число, значит дополнительный отбор дат не требуется

3. Запрос для решения задачи:

In [5]:
df_final = '''
select count(*) as cnt_stud_cor_20_or_more
    from(
        select st_id, sum(correct) as count_correct
        from default.peas
        group by st_id
        having count_correct >= 20
        ) as df
'''

print('Количество очень усердных студентов:', int(ph.execute(df_final, connection_default)))

Количество очень усердных студентов: 136


# В одном запросе выгрузить следующую информацию о группах пользователей:

ARPU 

ARPAU 

CR в покупку 

СR активного пользователя в покупку 

CR пользователя из активности по математике в покупку курса по математике

# Описание: 

Образовательная платформа предлагает пройти студентам курсы следующим образом: студент может решить бесплатно лишь 30 маленьких заданий в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Активным считаем пользователя, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считаем пользователя, который за все время решивший 2 или больше задач правильно по математике.

# Нахожу метрики по отдельности, соединяю с помощью CTE.

In [6]:
tack_2_ARPU = '''
select
    test_grp,
    round(sum(all_revenue/cnt_uniq_st), 2) as ARPU
from (
    select
        s.test_grp as test_grp,
        sum(f.money) as all_revenue
    from default.final_project_check as f
    left join default.studs as s on f.st_id=s.st_id
    group by test_grp
    ) as all_revenue
left join (
    select
        s.test_grp,
        uniqExact(p.st_id) as cnt_uniq_st
    from default.peas as p
    left join default.studs as s on p.st_id=s.st_id
    group by s.test_grp
    ) as cnt_uniq_st on all_revenue.test_grp=cnt_uniq_st.test_grp
group by test_grp
'''

ph.read_clickhouse(tack_2_ARPU, connection=connection_default)

,test_grp,ARPU
0,control,8393.94
1,pilot,23741.26


In [7]:
tack_2_ARPAU = '''
select
    test_grp,
    round(sum(n.avg_money/q.st_id), 2) as ARPAU
from (
    select
        test_grp,
        count(st_id) as st_id
    from (
        select
            ccm.st_id,
            ccm.cnt_correct_more10,
            s.test_grp
        from (
            select
                st_id,
                sum(correct) as cnt_correct_more10
            from default.peas
            group by st_id
            having cnt_correct_more10 > 10
            ) as ccm
        left join default.studs as s on ccm.st_id=s.st_id) as a
        group by test_grp
        ) as q
    left join (
        select
            test_grp,
            avg(money) as avg_money
        from (
            select 
                f.st_id,
                f.money,
                s.test_grp
            from default.final_project_check as f
            left join default.studs as s on f.st_id=s.st_id)
            group by test_grp
            ) as n on n.test_grp=q.test_grp
group by test_grp
'''
ph.read_clickhouse(tack_2_ARPAU, connection=connection_default)

,test_grp,ARPAU
0,control,605.86
1,pilot,822.43


In [8]:
tack_2_CR = '''
select
    r.test_grp,
    round(sum(buyers/all_st), 4) as CR
from (
    select
        s.test_grp,
        uniqExact(f.st_id) as buyers
    from default.final_project_check as f
    left join default.studs as s on f.st_id=s.st_id
    group by test_grp
    ) as r
left join (
    select
        s.test_grp,
        uniqExact(p.st_id) as all_st
    from default.peas as p
    left join default.studs as s on p.st_id=s.st_id
    group by test_grp
    ) as w on r.test_grp=w.test_grp
group by test_grp
'''

ph.read_clickhouse(tack_2_CR, connection=connection_default)

,test_grp,CR
0,control,0.0909
1,pilot,0.2238


In [9]:
tack_2_CR_active = '''
select
    test_grp,
    round((y.buyers/t.st_id), 4) as CR_active
from (
    select
        test_grp,
        count(st_id) as st_id
    from (
        select
            ccm.st_id,
            ccm.cnt_correct_more10,
            s.test_grp
        from (
            select
                st_id,
                sum(correct) as cnt_correct_more10
            from default.peas
            group by st_id
            having cnt_correct_more10 > 10
            ) as ccm
        left join default.studs as s on ccm.st_id=s.st_id) as a
        group by test_grp) as t
    left join (
        select
            s.test_grp,
            uniqExact(f.st_id) as buyers
        from default.final_project_check as f
        left join default.studs as s on f.st_id=s.st_id
        group by test_grp
        ) as y on t.test_grp=y.test_grp
'''
ph.read_clickhouse(tack_2_CR_active, connection=connection_default)

,test_grp,CR_active
0,control,0.1181
1,pilot,0.3333


In [10]:
tack_2_CR_math = '''
select
    test_grp,
    round((cnt_buy_math/cnt_math), 4) as CR_math
from (
    select
        s.test_grp,
        count(u.st_id) as cnt_math
    from (
        select 
            st_id,
            sum(correct) as cnt_correct_2_and_more
        from default.peas
        where subject = 'Math'
        group by st_id
        having cnt_correct_2_and_more >= 2
        ) as u 
    left join default.studs as s on u.st_id=s.st_id
    group by test_grp
    ) as i
left join (
    select
        test_grp,
        uniqExact(st_id) as cnt_buy_math
    from (
        select *
        from default.final_project_check as f
        left join default.studs as s on f.st_id=s.st_id
        where subject = 'Math'
        )
    group by test_grp) as l on l.test_grp=i.test_grp
'''
ph.read_clickhouse(tack_2_CR_math, connection=connection_default)

,test_grp,CR_math
0,control,0.1633
1,pilot,0.3810


In [11]:
# Решение задачи в одном запросе
tack_2 = '''
with arpu as (select
    test_grp,
    round(sum(all_revenue/cnt_uniq_st), 2) as ARPU
from (
    select
        s.test_grp as test_grp,
        sum(f.money) as all_revenue
    from default.final_project_check as f
    left join default.studs as s on f.st_id=s.st_id
    group by test_grp
    ) as all_revenue
left join (
    select
        s.test_grp,
        uniqExact(p.st_id) as cnt_uniq_st
    from default.peas as p
    left join default.studs as s on p.st_id=s.st_id
    group by s.test_grp
    ) as cnt_uniq_st on all_revenue.test_grp=cnt_uniq_st.test_grp
group by test_grp) ,

arpau as (select
    test_grp,
    round(sum(n.avg_money/q.st_id), 2) as ARPAU
from (
    select
        test_grp,
        count(st_id) as st_id
    from (
        select
            ccm.st_id,
            ccm.cnt_correct_more10,
            s.test_grp
        from (
            select
                st_id,
                sum(correct) as cnt_correct_more10
            from default.peas
            group by st_id
            having cnt_correct_more10 > 10
            ) as ccm
        left join default.studs as s on ccm.st_id=s.st_id) as a
        group by test_grp
        ) as q
    left join (
        select
            test_grp,
            avg(money) as avg_money
        from (
            select 
                f.st_id,
                f.money,
                s.test_grp
            from default.final_project_check as f
            left join default.studs as s on f.st_id=s.st_id)
            group by test_grp
            ) as n on n.test_grp=q.test_grp
group by test_grp),

cr_buy as (select
    r.test_grp,
    round(sum(buyers/all_st), 4) as CR
from (
    select
        s.test_grp,
        uniqExact(f.st_id) as buyers
    from default.final_project_check as f
    left join default.studs as s on f.st_id=s.st_id
    group by test_grp
    ) as r
left join (
    select
        s.test_grp,
        uniqExact(p.st_id) as all_st
    from default.peas as p
    left join default.studs as s on p.st_id=s.st_id
    group by test_grp
    ) as w on r.test_grp=w.test_grp
group by test_grp),

cr_active as (select
    test_grp,
    round((y.buyers/t.st_id), 4) as CR_active
from (
    select
        test_grp,
        count(st_id) as st_id
    from (
        select
            ccm.st_id,
            ccm.cnt_correct_more10,
            s.test_grp
        from (
            select
                st_id,
                sum(correct) as cnt_correct_more10
            from default.peas
            group by st_id
            having cnt_correct_more10 > 10
            ) as ccm
        left join default.studs as s on ccm.st_id=s.st_id) as a
        group by test_grp) as t
    left join (
        select
            s.test_grp,
            uniqExact(f.st_id) as buyers
        from default.final_project_check as f
        left join default.studs as s on f.st_id=s.st_id
        group by test_grp
        ) as y on t.test_grp=y.test_grp),

cr_math as (select
    test_grp,
    round((cnt_buy_math/cnt_math), 4) as CR_math
from (
    select
        s.test_grp,
        count(u.st_id) as cnt_math
    from (
        select 
            st_id,
            sum(correct) as cnt_correct_2_and_more
        from default.peas
        where subject = 'Math'
        group by st_id
        having cnt_correct_2_and_more >= 2
        ) as u 
    left join default.studs as s on u.st_id=s.st_id
    group by test_grp
    ) as i
left join (
    select
        test_grp,
        uniqExact(st_id) as cnt_buy_math
    from (
        select *
        from default.final_project_check as f
        left join default.studs as s on f.st_id=s.st_id
        where subject = 'Math'
        )
    group by test_grp) as l on l.test_grp=i.test_grp)
    
    select
        a.test_grp as "group",
        a.ARPU as "ARPU",
        b.ARPAU as "ARPAU",
        c.CR as "CR",
        e.CR_active as "CR_active",
        d.CR_math as "CR_math"
    from arpu as a 
    left join arpau as b on a.test_grp = b.test_grp
    left join cr_buy as c on a.test_grp = c.test_grp
    left join cr_math as d on a.test_grp = d.test_grp
    left join cr_active as e on a.test_grp = e.test_grp
'''
ph.read_clickhouse(tack_2, connection=connection_default)

,group,ARPU,ARPAU,CR,CR_active,CR_math
0,control,8393.94,605.86,0.0909,0.1181,0.1633
1,pilot,23741.26,822.43,0.2238,0.3333,0.3810
